# Highlighter plot
**Vivek Hariharan, 2024-01-09**

This R script is used to compare a set of DNA sequences to a reference sequence and visualize the differences using a plot.

For the sake of this Jupyter markdown, I will suppress all outputs, but example outputs can be found in the README.md file.

First we will install and load the required packages.

In [ ]:
# Install and load necessary packages
if (!requireNamespace("BiocManager", quietly = TRUE))
  
        install.packages("BiocManager")
BiocManager::install("Biostrings")

install.packages("ggplot2")
library(Biostrings)
library(ggplot2)


Next, we will read in the FASTA file, specify the reference sequence, and create a dataframe to store the information about each sequence.
We will add a column to the dataframe representing the "Type". 
First, we will call any deletions.  
Next we will retain any differences from the reference.
Any matches to the reference will be called "Match".

In [ ]:

# Read the sequences from the FASTA file
sequences <- readDNAStringSet("All484.fas")

# Specify the reference sequence
ref_seq <- 3 #Change this to your actual sequence number
reference_sequence <- sequences[ref_seq]

# Create a data frame to store information about each position in each sequence
plot_data <- do.call(rbind, lapply(1:length(sequences), function(i) {
    df <- data.frame(
        Sequence = i,
        Position = 1:width(sequences[i]),
        Nucleotide = strsplit(as.character(sequences[i]), '')[[1]],
        Reference = strsplit(as.character(reference_sequence), '')[[1]]
    )
    
    # If it's a deletion, label it as 'Deletion'
    df$Type[df$Nucleotide == "-"] <- "Deletion"
    
    # If it's different from the reference, copy the difference
    df$Type[df$Nucleotide != df$Reference & is.na(df$Type)] <- df$Nucleotide[df$Nucleotide != df$Reference & is.na(df$Type)]
    
    # If it's the same as the reference, label it as 'Match'
    df$Type[is.na(df$Type)] <- "Match"
    
    return(df)
}))


Now that all the information in in the "Type" column of the dataframe, we are almost done.

Next we change the "Type" of the reference sequence to "Reference".
Then we will set the sequencing bounds. This is custom and based on the amplicon itself. This must be changed for every unique amplicon.



In [ ]:

#Make reference sequence plot_data$type be "reference"
plot_data$Type[plot_data$Sequence == ref_seq] <- "Reference"

#Set sequencing bounds for the data. In this case we are seq HXB2 coord 670-9690
# For sequences 2-end, make the plot_data$type be "notseq" from nucleotides 1-670 and 9690-end
plot_data$Type[plot_data$Sequence != 1 & plot_data$Position <= 670] <- "notseq"
plot_data$Type[plot_data$Sequence != 1 & plot_data$Position >= 9690] <- "notseq"


# Calculate the maximum position before plotting
max_position <- max(plot_data$Position)
max_length <- max(width(sequences))

Last is plotting.
Currently the plot is using  
```{r} 
geom_tile() 
``````

function from ggplot2, which plots each sequence as a collection of N tiles, where N is the number of bases in the sequence.


In [ ]:
# Create the plot
p <- ggplot(plot_data, aes(x = Position, y = as.numeric(Sequence), fill = Type)) +
    # Lines that go across each sequence
    annotate("segment",
        x = rep(0, length(sequences)),
        xend = rep(max_position, length(sequences)),
        y = 1:length(sequences),
        yend = 1:length(sequences),
        color = "gray",
        size = 0.2,
        alpha = 0.4) +
    geom_tile(height = 0.8) +
    #scale_x_continuous(breaks = seq(1, max_position, 1000)) +
    scale_y_reverse(breaks = seq(1, length(sequences), 10)) +
    scale_fill_manual(values = c(
        "T" = "red",
        "A" = "green",
        "C" = "blue",
        "G" = "black", 
        "Match" = rgb(0, 0, 0, 0),
        "Deletion" = "gray",
        "notseq" = rgb(0, 0, 0, 0),
        "Reference" = rgb(0, 0, 0, 0)),
        breaks = c("T", "A", "C", "G", "Deletion", "Reference")) +
    theme_minimal() +
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()) + # Remove grid lines  
    ylab('Sequence') +
    xlab('Position (HXB2 Coordinates)') +
    ggtitle('Highlighter Plot') +
    theme(panel.border = element_rect(colour = "black", fill=NA, linewidth=0.5)) +
    theme(axis.text.y = element_text(angle = 0, color = "black"), # Change Y axis text
          axis.text.x = element_text(color = "black")) + # Change X axis text
    theme(axis.text.y = element_text(angle = 0)) +
    theme(axis.ticks = element_line(color = "black"))

# Save the plot as a PDF file with 600dpi
ggsave("highlighter_plot.png", plot = p, device = "png", dpi = 600)
